In [6]:
""" 
Author : ronald turyatemba
Date : 24th october 2016


"""


import csv as csv
import numpy as np

csv_file_object = csv.reader(open('train.csv', 'rt'))       # Load in the csv file
header = next(csv_file_object)                             # Skip the fist line as it is a header
data=[]                                                     # Create a variable to hold the data

for row in csv_file_object:                 # Skip through each row in the csv file
    data.append(row)                        # adding each row to the data variable
data = np.array(data)                       # Then convert from a list to an array

# In order to analyse the price column I need to bin up that data
# here are my binning parameters, the problem we face is some of the fares are very large
# So we can either have a lot of bins with nothing in them or we can just lose some
# information by just considering that anythng over 39 is simply in the last bin.
# So we add a ceiling
fare_ceiling = 40
# then modify the data in the Fare column to = 39, if it is greater or equal to the ceiling
data[ data[0::,9].astype(np.float) >= fare_ceiling, 9 ] = fare_ceiling - 1.0

fare_bracket_size = 10
number_of_price_brackets = fare_ceiling // fare_bracket_size
#number_of_classes = 3                             # I know there were 1st, 2nd and 3rd classes on board.
number_of_classes = len(np.unique(data[0::,2]))   # But it's better practice to calculate this from the Pclass directly:
                                                  # just take the length of an array of UNIQUE values in column index 2
print(number_of_classes)

# This reference matrix will show the proportion of survivors as a sorted table of
# gender, class and ticket fare.
# First initialize it with all zeros
survival_table = np.zeros([2,number_of_classes,number_of_price_brackets],float)

# I can now find the stats of all the women and men on board
for i in range(number_of_classes):
    for j in range(number_of_price_brackets):

        women_only_stats = data[ (data[0::,4] == "female") \
                                 & (data[0::,2].astype(np.float) == i+1) \
                                 & (data[0:,9].astype(np.float) >= j*fare_bracket_size) \
                                 & (data[0:,9].astype(np.float) < (j+1)*fare_bracket_size), 1]

        men_only_stats = data[ (data[0::,4] != "female") \
                                 & (data[0::,2].astype(np.float) == i+1) \
                                 & (data[0:,9].astype(np.float) >= j*fare_bracket_size) \
                                 & (data[0:,9].astype(np.float) < (j+1)*fare_bracket_size), 1]

                                 #if i == 0 and j == 3:

        survival_table[0,i,j] = np.mean(women_only_stats.astype(np.float))  # Female stats
        survival_table[1,i,j] = np.mean(men_only_stats.astype(np.float))    # Male stats

# Since in python if it tries to find the mean of an array with nothing in it
# (such that the denominator is 0), then it returns nan, we can convert these to 0
# by just saying where does the array not equal the array, and set these to 0.
survival_table[ survival_table != survival_table ] = 0.

# Now I have my proportion of survivors, simply round them such that if <0.5
# I predict they dont surivive, and if >= 0.5 they do
survival_table[ survival_table < 0.5 ] = 0
survival_table[ survival_table >= 0.5 ] = 1

# Now I have my indicator I can read in the test file and write out
# if a women then survived(1) if a man then did not survived (0)
# First read in test
test_file = open('test.csv', 'rt')
test_file_object = csv.reader(test_file)
header = next(test_file_object)

# Also open the a new file so I can write to it. 
predictions_file = open("genderclassmodel.csv", "wt")
predictions_file_object = csv.writer(predictions_file)
predictions_file_object.writerow(["PassengerId", "Survived"])

# First thing to do is bin up the price file
for row in test_file_object:
    for j in range(number_of_price_brackets):
        # If there is no fare then place the price of the ticket according to class
        try:
            row[8] = float(row[8])    # No fare recorded will come up as a string so
                                      # try to make it a float
        except:                       # If fails then just bin the fare according to the class
            bin_fare = 3 - float(row[1])
            break                     # Break from the loop and move to the next row
        if row[8] > fare_ceiling:     # Otherwise now test to see if it is higher
                                      # than the fare ceiling we set earlier
            bin_fare = number_of_price_brackets - 1
            break                     # And then break to the next row

        if row[8] >= j*fare_bracket_size\
            and row[8] < (j+1)*fare_bracket_size:     # If passed these tests then loop through
                                                      # each bin until you find the right one
                                                      # append it to the bin_fare
                                                      # and move to the next loop
            bin_fare = j
            break
        # Now I have the binned fare, passenger class, and whether female or male, we can
        # just cross ref their details with our survival table
    if row[3] == 'female':
        predictions_file_object.writerow([row[0], "%d" % int(survival_table[ 0, float(row[1]) - 1, bin_fare ])])
    else:
        predictions_file_object.writerow([row[0], "%d" % int(survival_table[ 1, float(row[1]) - 1, bin_fare])])

# Close out the files
test_file.close()
predictions_file.close()

3


c:\users\admin\appdata\local\programs\python\python35-32\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
c:\users\admin\appdata\local\programs\python\python35-32\lib\site-packages\numpy\core\_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\admin\appdata\local\programs\python\python35-32\lib\site-packages\ipykernel\__main__.py:102: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
c:\users\admin\appdata\local\programs\python\python35-32\lib\site-packages\ipykernel\__main__.py:100: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [6]:
import csv as csv
import numpy as np

csv_file_object = csv.reader(open('train.csv','rt'))
header = next(csv_file_object)
data = []

for row in csv_file_object:
    data.append(row)

data = np.array(data)

#print(data)

#print(data[0:15,5])

#i am going to find the mean of the age column values
#but first i should check the type of value this slice from the array is

type(data[0:,5])
#we can only find the mean of foats or integers here
#am gona convert all the values in this column to floats and save them in another array called all_ages
#all_ages = data[0:,5].astype(float)
#print(all_ages)
#we are getting an error here because there empty values in the selection, 
#so we are gona use pandas to clean the data, which means opening the train
#file all again using pandas methods and functions

import pandas as pd 
#using pd object
df = pd.read_csv('train.csv', header=0)
#to view the dataframe
#df
type(df)
#there is very useful information in here
df.info()
#as you can see from the script before, 891 are non null objects ie they have no
#non values, but for age and cabins, they have some non values

#pandas can also help us calculate important mathematical values such as 
#min max mean etc. It will skip the empty values
df.describe()
df['Age'][0:10]
df.Age[0:10]

#we finding the mean of the age
df['Age'].mean()
df[df['Age']>60]
#what the older passengers look like
#df[df['Age'] > 60][['Sex', 'Pclass', 'Age','Survived']]

#so now we are going to filter out the missing value, rows with null values
#so the script below will display all these 
df[df['Age'].isnull()][['Sex','Pclass','Age']]

#let try the combined & in sscripting
#e.g we are printing the number of males in each class

for i in range(1,4):
    #print (i, len(df[ (df['Sex'] == 'male') & (df['Pclass'] == i) ]))
    print (i, \
    len(df[(df['Sex'] =="male") & (df['Pclass'] == i)]))
    
#coming up a histogram of the ages, 
#Using the describe function, i will get the max of age which is 80 and min approx 0
df.describe()

#so the range is between 0 to 80 and a gap of 5 to narrow the scale
#import the libs first
#WE USING THE DROPNA FUNCTION TO DROP THE NULL VALUES
import pylab as p
df['Age'].dropna().hist(range = (0,80), alpha=.5)
p.xlabel('AGES')
p.ylabel('NUMBER OF PEOPLE')
#p.show()
#remeber if we had two lists to plot
#it  would be like so
#p.plot(list1 , list2)
#p.show...this plots list2 on x axis aginst list2 on yaxis 

#df['Gender'] = 4
#df.head()

#df['Gender'] = df['Sex'].map(lambda x: x[0].upper())
df['Gender'] = df['Sex'].map( {'female':0, 'male':1} ).astype(int)
df.head()
#we are now gona use this, for the rows with null values in ages , we shall put the median of the class ages that class belongs
#we are getting the median of @ gender in each class , ie median of females in first class, median of females in second class....
#first the array is set to zeros
median_ages = np.zeros((2,3))
median_ages

#then am going to populate this array with values
for i in range(0,2):
    for j in range (0,3):
        median_ages[i,j] = df[(df['Gender']==i) & (df['Pclass']==j+1)]['Age'].dropna().median()


median_ages    

#gona create another column and duplicate in the values of the age column into that column
df['AgeFill'] = df['Age']
#df.head()
#we are gona fill in the null values in AgeFill based on the median_ages table
#for every row that has no value for age is df['Age'].isnull(), we shall give it the median of that group from the table

#df[ df['Age'].isnull() ][['Gender','Pclass','Age','AgeFill']].head(10)

for i in range(0,2):
    for j in range(0,3):
#note that am using loc because it is more intuitive and less confusing for 
#pandas to make a slice selection suiting all the conditions
#other wise df[df[..]..] wwould work but may return messy results
        #df.loc[(df.Gender ==i)& (df.Pclass ==j+1)&(df.Age.isnull())] = median_ages[i,j]
        #note the above script only resulted into a table without values so note that the order of the conditions matter
        
        df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1),\
                'AgeFill'] = median_ages[i,j]
#lets print to see if the null values were subsitituted for medians        
df[df['Age'].isnull()][['Gender','Pclass','Age','AgeFill']].head(10)    

#am gona add one more column to see if the row in that column was changed
#from null to what it is
#df['AgeIsNull'] = df['Age'].isnull().astype(int)

#df.head(20)

#we are gona add afew more feature that we think will be usefull during the machine learning
#we know sibsp is sibling or spouse and parch is parent or child
df['FamilySize'] = df['SibSp'] + df['Parch']

#aalso
#age and class were factors for survival as third class old people were less likey to survive
#we have used a new syntax again , probably shall be using 
#when dealing with comparatives and numerical operators
df['Age*Class'] = df.AgeFill * df.Pclass

#note that to amply machine learning in on this data 
#1.all columns should not have a string 
#2.we should convert the pandas data frame into numerical python
# using dtypes() we shall find out which columns are 'objects' these are strings in pandas

df.dtypes
#or we could use this to show only the columns that are objects
df.dtypes[df.dtypes.map(lambda x: x=='object')]

#now we drop all columns that we shall not need
del df['Name']
#Or we use this better script
df = df.drop(['Ticket','Sex','Cabin','Embarked','PassengerId'], axis=1)
#we are going to delete the age column because we now have agefill column that is clean

df = df.drop(['Age'], axis=1)

#alternatively we could have dropped all rows that have missing values
#But be carefull with it coz if the row has any null value anywhere, it will be deleted
#so you have to be sure of the dataset
df = df.dropna()
#finally lets convert this into a numpy array using the .values function
train_data = df.values
#train_data
#df

#Prepare the test data

test_df = pd.read_csv('test.csv', header=0)        # Load the test file into a dataframe

# I need to do the same with the test data now, so that the columns are the same as the training data
# I need to convert all strings to integer classifiers:
# female = 0, Male = 1
test_df['Gender'] = test_df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

# Embarked from 'C', 'Q', 'S'
# All missing Embarked -> just make them embark from most common place
#if len(test_df.Embarked[ test_df.Embarked.isnull() ]) > 0:
#    test_df.Embarked[ test_df.Embarked.isnull() ] = test_df.Embarked.dropna().mode().values
# Again convert all Embarked strings to int
#test_df.Embarked = test_df.Embarked.map( lambda x: Ports_dict[x]).astype(int)

#we are gona add afew more feature that we think will be usefull during the machine learning
#we know sibsp is sibling or spouse and parch is parent or child
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch']

#aalso

# All the ages with no data -> make the median of all Ages
median_age = test_df['Age'].dropna().median()
if len(test_df.Age[ test_df.Age.isnull() ]) > 0:
    test_df.loc[ (test_df.Age.isnull()), 'Age'] = median_age

#age and class were factors for survival as third class old people were less likey to survive
#we have used a new syntax again , probably shall be using 
#when dealing with comparatives and numerical operators
test_df['Age*Class'] = test_df.Age * test_df.Pclass

# All the missing Fares -> assume median of their respective class
if len(test_df.Fare[ test_df.Fare.isnull() ]) > 0:
    median_fare = np.zeros(3)
    for f in range(0,3):                                              # loop 0 to 2
        median_fare[f] = test_df[ test_df.Pclass == f+1 ]['Fare'].dropna().median()
    for f in range(0,3):                                              # loop 0 to 2
        test_df.loc[ (test_df.Fare.isnull()) & (test_df.Pclass == f+1 ), 'Fare'] = median_fare[f]

# Collect the test data's PassengerIds before dropping it
ids = test_df['PassengerId'].values


# Remove the Name column, Cabin, Ticket, and Sex (since I copied and filled it to Gender)
test_df = test_df.drop(['Name', 'Sex','Ticket', 'Cabin', 'PassengerId','Embarked'], axis=1) 


#lets begin with randomforest on this array, note that, randomforest
#in only applicable on float values

from sklearn.ensemble import RandomForestClassifier


#create a random forest that includes all the parameters
forest = RandomForestClassifier(n_estimators=100)

#fit the data, this builds a forest from this set of data where the
#first array is the data on the x axis and the second array is the data- y axis
forest = forest.fit(train_data[0:,1:], train_data[0:,0])

#take the same decision trees and run it on the test data
#first get the numpy array of the test_df
#test_df
#df
test_data = test_df.values
output = forest.predict(test_data.astype(int))

df

#gona right the array into a csv file
randomforest = open('randomforest.csv', 'wt')
randomforest_object = csv.writer(randomforest)
randomforest_object.writerow(['PassengerId','Survived'])

print('done')
#output
#write each value of the output array into a row
number_of_rows = len(output)
for i in range(number_of_rows):
    randomforest_object.writerow([i,output[i]])
    
randomforest.close()    




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
1 122
2 108
3 347


C:\Users\Admin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


done
